In [4]:
import numpy as np
from scipy.special import expit, logit
from scipy import signal
import tensorflow as tf

def mse(y_test, y_pred):
    return np.mean(np.power(y_test - y_pred, 2))/2

def mse_deriv(y_test, y_pred):
    return (y_pred - y_test) / len(y_test)

def cross_entropy(y_test, y_pred):
    return np.mean(-y_test * np.log(y_pred) - (1 - y_test) * np.log(1 - y_pred))

def cross_entropy_deriv(y_test, y_pred):
    return ((1 - y_test) / (1 - y_pred) - y_test / y_pred) / len(y_test)

In [3]:
def sigmoid(x):
    return expit(x)
def tahn(x):
    return np.tanh(x)    
def relu(x):
    return np.maximum(0,x)
def linear(x):
    return x
def deriv_sigmoid(x):
    return expit(x)*(1-expit(x))
def deriv_tahn(x):
    return -4/(np.exp(x)-np.exp(-x))**2
def deriv_relu(x):
    return (x>0)*1
def deriv_linear(x):
    return 1

In [4]:
class Activation():
    def __init__(self, activation_function_name):
        self.activation_functions = dict(sigmoid=sigmoid,tahn=tahn,relu=relu, linear=linear)
        self.deriv_functions = dict(sigmoid=deriv_sigmoid,tahn=deriv_tahn,relu=deriv_relu, linear=deriv_linear) 
        self.activation_function = self.activation_functions[activation_function_name]
        self.activation_function_deriv = self.deriv_functions[activation_function_name]

    def forward(self, input):
        self.output = self.activation_function(input)
        return self.output

    def backward(self, output_err, learning_rate):
        return np.multiply(output_err, self.activation_function_deriv(self.output))

In [5]:
class Dense():
    def __init__(self, input_shape, output_size):
        self.weights = np.random.rand(output_size, input_shape)
        self.b_coefs = np.random.rand(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.b_coefs

    def backward(self, output_err, learning_rate):
        dW = np.dot(output_err, self.input.T)
        backward_err = np.dot(self.weights.T, output_err)
        self.weights -= learning_rate * dW
        self.bias -= learning_rate * output_err
        return backward_err

In [6]:
class Convolutional():
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.rand(self.kernels_shape)
        self.kernels_b = np.random.rand(self.output_shape)

    def forward(self, input):
        self.input = input
        self.output = np.copy(self.kernels_b)
        for i in range(self.depth):
            for j in range(self.input_depth):
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i, j], "valid")
        return self.output

    def backward(self, output_err, learning_rate):
        kernels_err = np.zeros(self.kernels_shape)
        input_err = np.zeros(self.input_shape)

        for i in range(self.depth):
            for j in range(self.input_depth):
                kernels_err[i, j] = signal.correlate2d(self.input[j], output_err[i], "valid")
                input_err[j] += signal.convolve2d(output_err[i], self.kernels[i, j], "full")

        self.kernels -= learning_rate * kernels_err
        self.kernels_b -= learning_rate * output_err
        return input_err


In [7]:
class Reshape():
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, input):
        return np.reshape(input, self.output_shape)

    def backward(self, output_err, learning_rate):
        return np.reshape(output_err, self.input_shape)

In [8]:
class Neural_Network:
    def __init__(self):
        self.layers = []
        self.loss_functions = dict(mse = mse, cross_entropy = cross_entropy)
        self.deriv_loss_functions = dict(mse = mse_deriv, cross_entropy = cross_entropy_deriv)
    def add(self, layer):
        self.layers.append(layer)
    def compile(self, loss_function):
        self.loss_function = self.loss_functions[loss_function]
        self.loss_function_deriv = self.deriv_loss_functions[loss_function]
    def fit(self, X_train, y_train, epochs, learning_rate, verbose=True):
        for e in range(epochs):
            error = 0
            for x, y in zip(X_train, y_train):
                output = x
                for layer in self.layers:
                     output = layer.forward(output)

                error += self.loss_function(y, output)
                err = self.loss_function_deriv(y, output)
                for layer in reversed(self.layers):
                    err = layer.backward(err, learning_rate)

            error /= len(X_train)
            if verbose:
                print(f"{e + 1}/{epochs}, error={error}")

In [7]:
def predobrab_data(X):
    x_out, y_out = [], []
    for batch, labels in X:
        for img, label in zip(batch, labels):
            img = img.numpy()
            new_img = []
            for i in range(img.shape[2]):
                new_img.append(img[:,:,i])
            x_out.append(new_img)
            y_out.append(label)
    return x_out, y_out       

In [8]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    "../datasets/data3", # путь к изображениям
    validation_split=0.2, # процент на тест
    subset="both", # берем и обучающую и тестовую выборку
    seed=42, # сид генератора случайных чисел
    image_size=(32,32), # целевой размер изображения (СНАЧАЛА ВЫСОТА, ПОТОМ ШИРИНА)
    batch_size=32, # размер пакета (батча)
)

Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Using 400 files for validation.


In [9]:
X, y = predobrab_data(train_ds)
print(X[0])

[array([[ 89.21875 ,  63.609375, 101.765625, ..., 107.5     , 107.25    ,
        110.65625 ],
       [104.609375, 151.82812 , 106.34375 , ..., 136.625   ,  96.609375,
        105.28125 ],
       [106.015625, 106.375   ,  91.625   , ...,  71.90625 , 118.1875  ,
        110.65625 ],
       ...,
       [249.95312 , 245.84375 , 116.453125, ..., 120.8125  , 124.75    ,
        124.390625],
       [245.25    , 250.5     , 254.4375  , ..., 112.75    , 122.      ,
        119.1875  ],
       [255.      , 255.      , 255.      , ..., 112.578125, 112.953125,
        117.828125]], dtype=float32), array([[ 90.21875 ,  72.203125, 104.109375, ..., 105.71875 , 100.265625,
        111.65625 ],
       [101.609375, 153.14062 , 101.5     , ..., 126.453125,  97.609375,
        107.28125 ],
       [103.0625  , 105.375   ,  89.625   , ...,  64.90625 , 117.1875  ,
        102.796875],
       ...,
       [255.      , 255.      , 129.04688 , ..., 121.8125  , 125.75    ,
        127.140625],
       [255.      

In [19]:
print(1)

1
